In [1]:
from datasets import load_dataset
from training_datasets.GLUEDataset import GLUEDataset

cola = load_dataset('glue','cola')
# print(cola)
# sst2 = load_dataset('glue','sst2')
# print(sst2)
mrpc = load_dataset('glue', 'mrpc')
# print(mrpc)
pt_cola = GLUEDataset([cola, mrpc], 'train')

/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset glue (/home/aksingh/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|█████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 744.55it/s]
Found cached dataset glue (/home/aksingh/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 1551.91it/s]


In [2]:
from training_datasets.GLUEMetaDataset import GLUEMetaDataset
from samplers.FewShotEpisodeSampler import FewShotEpisodeSampler
from samplers.FewShotEpisodeBatchSampler import FewShotEpisodeBatchSampler
import torch.utils.data as data

train_protomaml_sampler = FewShotEpisodeBatchSampler(pt_cola, kShot=4, nWay=4, batchSize=4, shuffle=True)
train_protomaml_loader = data.DataLoader(
    pt_cola, batch_sampler=train_protomaml_sampler, collate_fn=train_protomaml_sampler.getCollateFunction(), num_workers=4)

# for i in range(1):
#     batch = next(iter(train_protomaml_loader))
#     for episode_i in range(len(batch[0])):
#         data, labels = batch[0][episode_i], batch[1][episode_i]
#         supportSet, supportLabels = data[0:len(data)//2], labels[0:len(data)//2] 
#         print(supportLabels)

In [3]:
# episodes = []
# for i in range(1):
#     episodes.append(gm_ds.getTask())
# for i in range(len(episodes)):
#     classes = len(set(episodes[i][1].tolist()))
#     # print(classes)
#     print(episodes[i][0], episodes[i][1], "\n")

In [4]:
# from training.models.ProtoFOMAML import ProtoFOMAML

# pfomaml = ProtoFOMAML(outerLR=5e-4, innerLR=1e-3, outputLR=1e-2, steps=5, batchSize=16, warmupSteps=0)
# pfomaml.training_step(next(iter(train_protomaml_loader)), 0)

In [5]:
from samplers.FewShotValidationEpisodeBatchSampler import FewShotValidationEpisodeBatchSampler
from samplers.FewShotValidationEpisodeSampler import FewShotValidationEpisodeSampler
from validation_datasets.ValidationDataset import ValidationDataset
import torch.utils.data as data

ds = ValidationDataset()

val_protomaml_sampler = FewShotValidationEpisodeBatchSampler(ds, kShot=2)
val_protomaml_loader = data.DataLoader(
    ds, batch_sampler=val_protomaml_sampler, collate_fn=val_protomaml_sampler.getCollateFunction(), num_workers=1
)

In [6]:
from training.models.ProtoFOMAML import ProtoFOMAML

# pfomaml = ProtoFOMAML(outerLR=5e-4, innerLR=1e-3, outputLR=1e-2, steps=5, batchSize=16, warmupSteps=0)
# pfomaml.validation_step(next(iter(val_protomaml_loader)), 0)

In [ ]:
from training.trainer import train_model

protomaml_model = train_model(
    ProtoFOMAML,
    train_loader=train_protomaml_loader,
    val_loader=val_protomaml_loader,
    outerLR=5e-4, innerLR=1e-3, outputLR=1e-2, steps=2, batchSize=4, warmupSteps=0
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassificat

final meta param is Parameter containing:
tensor([[ 0.0089, -0.0001,  0.0218,  ..., -0.0238,  0.0196, -0.0266],
        [-0.0358, -0.0254, -0.0093,  ...,  0.0072,  0.0148, -0.0325],
        [ 0.0181,  0.0223, -0.0153,  ...,  0.0151, -0.0040,  0.0030],
        ...,
        [ 0.0166, -0.0064, -0.0283,  ..., -0.0177, -0.0344,  0.0235],
        [ 0.0303, -0.0146, -0.0131,  ...,  0.0229,  0.0065, -0.0010],
        [ 0.0029, -0.0326, -0.0306,  ..., -0.0233, -0.0327, -0.0275]],
       requires_grad=True)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predic

inner loop training loss is 7.104135513305664
inner loop training loss is 6.306189775466919
inner loop training loss is 7.283535003662109
inner loop training loss is 6.1034533977508545
local param grad is tensor([[ 2.7868e-05, -6.8000e-03, -3.2965e-03,  ..., -1.3188e-03,
         -1.9701e-04,  1.2363e-03],
        [-1.7968e-03, -1.8249e-03, -5.0423e-04,  ...,  1.3661e-03,
         -1.0696e-03,  3.2288e-04],
        [ 8.7294e-03, -6.5624e-03, -3.7960e-03,  ..., -9.5473e-03,
          5.6266e-03,  1.0008e-03],
        ...,
        [ 3.0199e-04, -5.0021e-03, -2.5490e-03,  ..., -1.2396e-03,
         -3.4666e-05,  9.2328e-04],
        [ 8.1995e-04,  8.4366e-03,  4.0401e-03,  ...,  8.2386e-04,
          8.2346e-04, -1.5547e-03],
        [-1.6056e-04, -2.3330e-03, -1.0586e-03,  ..., -2.8963e-04,
         -1.3614e-04,  4.1628e-04]])
local param grad is tensor([[ 3.6178e-03, -4.4222e-03, -7.0917e-04,  ..., -7.0518e-04,
          1.6705e-03,  5.0263e-03],
        [-3.7669e-04, -9.4205e-04,  9.47

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predic

...

inner loop training loss is 4.5267369747161865
inner loop training loss is 3.837843656539917
inner loop training loss is 3.8704177141189575
inner loop training loss is 3.2564550638198853
local param grad is tensor([[-8.6000e-03, -7.8290e-03, -3.6141e-03,  ...,  9.6875e-03,
         -1.2555e-02,  3.3313e-03],
        [-4.1701e-03, -1.6294e-03, -9.4679e-05,  ...,  3.1144e-03,
         -3.7885e-03,  6.5396e-04],
        [ 8.9007e-03,  1.1855e-02,  6.7530e-03,  ..., -1.2947e-02,
          1.7344e-02, -5.1081e-03],
        ...,
        [ 1.0539e-02,  2.0416e-02,  1.2967e-02,  ..., -2.0105e-02,
          2.7541e-02, -8.8754e-03],
        [-1.6495e-02, -6.6283e-03, -2.5479e-04,  ...,  1.2124e-02,
         -1.4505e-02,  2.6765e-03],
        [ 1.2491e-03,  4.2908e-03,  2.9528e-03,  ..., -3.7303e-03,
          5.2098e-03, -1.8825e-03]])
local param grad is tensor([[-9.2806e-03, -3.6711e-03, -1.6902e-03,  ...,  5.1220e-04,
         -1.1739e-02,  1.6926e-03],
        [-4.3756e-03, -7.5913e-04,  2.

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predic

inner loop training loss is 4.13526725769043
inner loop training loss is 2.624081254005432
inner loop training loss is 5.032874822616577
inner loop training loss is 3.1256706714630127
local param grad is tensor([[-1.9592e-04, -2.3190e-03, -3.0792e-06,  ...,  8.8723e-03,
         -8.0828e-04,  1.7427e-03],
        [-6.8935e-03, -3.8352e-03,  6.5404e-03,  ...,  7.5024e-03,
         -1.5395e-04,  1.5162e-03],
        [ 5.9820e-04,  3.5801e-03, -1.1798e-03,  ..., -1.2617e-03,
         -2.6713e-03, -2.6470e-03],
        ...,
        [-6.8878e-03,  2.7523e-03,  7.0600e-03,  ..., -1.7785e-02,
          2.0763e-03, -3.5495e-03],
        [-2.9330e-03,  4.7326e-04,  3.0523e-03,  ..., -6.2744e-03,
          1.0892e-03, -9.7197e-04],
        [ 9.1459e-03, -1.6056e-03, -9.0891e-03,  ...,  1.4088e-02,
         -1.5330e-03,  3.1397e-03]])
local param grad is tensor([[-4.9826e-04, -2.9846e-03, -3.6418e-07,  ...,  5.2401e-03,
          1.6125e-03,  3.8174e-03],
        [-5.8492e-03, -4.8285e-03,  5.101

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predic

inner loop training loss is 5.5324554443359375
inner loop training loss is 5.977449178695679
inner loop training loss is 4.3144142627716064
inner loop training loss is 5.157185077667236
local param grad is tensor([[-8.6027e-04, -1.2946e-05,  1.9647e-03,  ...,  1.1356e-03,
         -1.5504e-03,  1.8789e-04],
        [-4.7940e-03, -1.6281e-04,  1.1344e-02,  ...,  6.4761e-03,
         -8.9759e-03,  1.1966e-03],
        [ 2.7979e-03,  5.2563e-05, -6.4343e-03,  ..., -3.7098e-03,
          5.0802e-03, -6.2799e-04],
        ...,
        [ 9.6268e-04,  1.2845e-05, -2.1898e-03,  ..., -1.2673e-03,
          1.7275e-03, -2.0713e-04],
        [-2.4983e-03, -7.9933e-05,  5.8886e-03,  ...,  3.3660e-03,
         -4.6578e-03,  6.1502e-04],
        [ 3.6293e-03,  1.6793e-04, -8.7885e-03,  ..., -4.9783e-03,
          6.9657e-03, -9.8101e-04]])
local param grad is tensor([[-0.0005, -0.0010,  0.0012,  ...,  0.0007, -0.0010,  0.0003],
        [-0.0024, -0.0061,  0.0070,  ...,  0.0039, -0.0058,  0.0018],
  